# 🦙 Instruction Fine-Tuning LLaMA-3.2-1B-Instruct with Chomsky Dataset

This notebook helps you fine-tune LLaMA-3.2-1B-Instruct (4-bit) using enriched question-answer data from Chomsky, using instruction-style tuning and LoRA (PEFT).

In [1]:
# ✅ Install required libraries
!pip install -q transformers datasets peft accelerate bitsandbytes

In [2]:
# 📥 Load and format dataset
import json
from datasets import Dataset

# Load your enriched dataset (place it in the same folder as this notebook)
with open("enriched_data_merged.json", "r", encoding="utf-8") as f:
    data = json.load(f)

def format_example(example):
    context = f"Year: {example['year']} | Setting: {example['setting']} | Persona: {example['persona']} | Topic: {example['topic']}"
    return {
        "text": f"""### Context:
{context}

### Instruction:
{example['instruction']}

### Input:
{example['input']}

### Response:
{example['output']}"""
    }

formatted_data = [format_example(d) for d in data]
dataset = Dataset.from_list(formatted_data)

In [3]:
# 🧠 Load model in 4-bit
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import prepare_model_for_kbit_training
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"  # Change to 1B variant if you have local access

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)

model = prepare_model_for_kbit_training(model)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct.
401 Client Error. (Request ID: Root=1-67f55015-0663c9a80506a64319d037ae;5bacd9c1-e2a2-4a8e-9381-ffaea51b5a82)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
# 🪝 LoRA config
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# ✂️ Tokenize dataset
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True)

In [ ]:
# 🏃 Train the model
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./llama3_chomsky_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    logging_steps=10,
    learning_rate=2e-4,
    bf16=True,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()

In [ ]:
# 💾 Save model
model.save_pretrained("./llama3_chomsky_lora")
tokenizer.save_pretrained("./llama3_chomsky_lora")

In [ ]:
# ✅ Inference
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = """### Context:
Year: 2020 | Setting: Oxford Debate | Persona: Historian | Topic: Humanities

### Instruction:
Respond as a historian in a Oxford Debate discussing Humanities

### Input:
How do you view the significance of this moment in human history?

### Response:
"""

output = pipe(prompt, max_new_tokens=150, do_sample=True, temperature=0.7)[0]["generated_text"]
print(output)